In [1]:
import interfaces as inter

In [3]:
file = 'TEST.txt'
inter.file_to_n_files(file,4) #will not work now, functions should be implemented first!

In [9]:
#t = %timeit -o !mpiexec —allow-run-as-root -n 4 python main.py
#will not work now, functions should be implemented first!
times = []
for n_use in [2000]:
    for n_keep in [1000]:
        for n_final_top in [500]:
            f = open('main.py','w')
            f.write('''import interfaces as inter
from mpi4py import MPI

N_USE = n_use
N_KEEP = n_keep
N_FINAL_TOP = n_final_top
FILENAME = 'TEST'

comm = MPI.COMM_WORLD
rank = comm.Get_rank()
filename = FILENAME + '_' + str(rank + 1) + '.txt'

inter.pipeline(filename, N_USE, N_KEEP, rank, comm,N_FINAL_TOP)'''.replace('n_use',str(n_use)).replace('n_keep',str(n_keep)).replace('n_final_top',str(n_final_top)))
            f.close()
            t = %timeit -o !mpiexec -n 4 python main.py
            times.append(t)
            #later we will add score calculation here

